In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.8 MB/s eta 0:00:00


In [5]:
!pip install --upgrade fsspec

  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.


In [7]:
!pip install -q datasets albumentations

In [38]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 5.6 MB/s eta 0:00:00


In [1]:
from transformers import MaskFormerForInstanceSegmentation

# Load MaskFormer model for semantic segmentation
model = MaskFormerForInstanceSegmentation.from_pretrained(
    "facebook/maskformer-swin-tiny-ade",
    num_labels=6,  # Number of labels for the dataset
    ignore_mismatched_sizes=True
)
model.to("cuda")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of MaskFormerForInstanceSegmentation were not initialized from the model checkpoint at facebook/maskformer-swin-tiny-ade and are newly initialized because the shapes did not match:
- class_predictor.bias: found shape torch.Size([151]) in the checkpoint and torch.Size([7]) in the model instantiated
- class_predictor.weight: found shape torch.Size([151, 256]) in the checkpoint and torch.Size([7, 256]) in the model instantiated
- criterion.empty_weight: found shape to

MaskFormerForInstanceSegmentation(
  (model): MaskFormerModel(
    (pixel_level_module): MaskFormerPixelLevelModule(
      (encoder): MaskFormerSwinBackbone(
        (model): MaskFormerSwinModel(
          (embeddings): MaskFormerSwinEmbeddings(
            (patch_embeddings): MaskFormerSwinPatchEmbeddings(
              (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
            )
            (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (encoder): MaskFormerSwinEncoder(
            (layers): ModuleList(
              (0): MaskFormerSwinStage(
                (blocks): ModuleList(
                  (0-1): 2 x MaskFormerSwinLayer(
                    (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
                    (attention): MaskFormerSwinAttention(
                      (self): MaskFormerSwinSelfAttention(
                        (query): Linear(in_fea

In [2]:
import os
import torch

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from transformers import MaskFormerImageProcessor


from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset

In [23]:
from PIL import Image
import torch
from torch.utils.data import Dataset
import torch.nn.functional as F
import os
import numpy as np

class SemanticSegmentationDataset(Dataset):
    """Image (semantic) segmentation dataset resized to 100x100 resolution."""

    def __init__(self, image_processor, txt_file, img_dir, masks_dir, target_size=(100, 100)):
        """
        Args:
            image_processor: The MaskFormer image processor.
            txt_file (str): Path to the .txt file containing filenames.
            img_dir (str): Base directory containing the images.
            masks_dir (str): Base directory containing the masks.
            target_size (tuple): Target resolution for images and masks.
        """
        self.image_processor = image_processor
        self.target_size = target_size  # Fixed resolution

        # Load filenames from txt file
        with open(txt_file, "r") as f:
            self.filenames = [line.strip() for line in f.readlines()]

        # Directories for images and masks
        self.img_dir = img_dir
        self.masks_dir = masks_dir

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        # Dynamically construct paths
        filename = self.filenames[idx]
        image_path = os.path.join(self.img_dir, filename)
        mask_path = os.path.join(self.masks_dir, filename.replace(".jpg", ".npy"))

        # Load image and mask
        image = Image.open(image_path).convert("RGB")
        mask = np.load(mask_path)

        # Resize image
        image_resized = image.resize(self.target_size, Image.BILINEAR)

        # Resize mask
        mask_resized = F.interpolate(
            torch.tensor(mask, dtype=torch.float32).unsqueeze(0).unsqueeze(0),
            size=self.target_size,
            mode="nearest"
        ).squeeze(0).squeeze(0).long()

        # Process the resized image
        encoded_inputs = self.image_processor(
            images=image_resized,
            return_tensors="pt",
            do_resize=False  # Prevent additional resizing
        )

        return {
            "pixel_values": encoded_inputs["pixel_values"].squeeze(0),  # Remove batch dimension
            "labels": mask_resized
        }

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import json
annotations_file ='/content/drive/My Drive/data/annotations.json'
with open(annotations_file, 'r') as f:
    annotations = json.load(f)
labels = []
for annotation in annotations:
    for shape in annotation['shapes']:
        labels.append(shape['label'])

labels = list(set(labels))
id_to_label = dict( zip(labels, list(range(1, len(labels) + 1))))

masks

In [25]:
import supervision as sv
from pathlib import Path

# Define input and output paths
data_dir = Path("/content/drive/My Drive/data")
output_dir = Path("/content/drive/My Drive/data/masks")

# Ensure the masks directory exists
output_dir.mkdir(parents=True, exist_ok=True)

# Get list of image files
images = sorted(list(data_dir.glob("imgs/*.jpg")))

# Assuming you have `annotations` and `id_to_label` variables
for idx, image_path in enumerate(images):
    mask = np.zeros((2662, 4004), dtype=np.uint8)  # Initialize blank mask

    # Extract filename without extension
    fname = image_path.stem  # Automatically extracts file stem (name without extension)

    for annotation in annotations:
        for shape in annotation['shapes']:
            if shape['frame'] == idx:
                if shape['type'] == 'polygon':
                    label = id_to_label[shape['label']]
                    points = np.array(shape['points'], dtype=np.float32).reshape((-1, 2))
                    polygon_mask = sv.polygon_to_mask(points, (4004, 2662))
                    polygon_mask[polygon_mask == 1] = label
                    mask = np.maximum(mask, polygon_mask)

    # Save the mask to the output directory
    save_path = output_dir / f"{fname}.npy"
    np.save(str(save_path), mask)

print("Masks successfully saved!")

Masks successfully saved!


In [26]:
image_processor = MaskFormerImageProcessor(ignore_index=255, reduce_labels=False)

# Base directories for images and masks
img_dir = '/content/drive/My Drive/data/imgs'   # Directory with images
masks_dir = '/content/drive/My Drive/data/masks'  # Directory with masks

# Paths to train.txt and test.txt files
train_txt = '/content/drive/My Drive/data/train.txt'
eval_txt = '/content/drive/My Drive/data/test.txt'

# Create train and evaluation datasets
train_ds = SemanticSegmentationDataset(
    image_processor=image_processor,
    txt_file=train_txt,
    img_dir=img_dir,
    masks_dir=masks_dir,
    target_size=(100, 100)  # Resize to 100x100
)

eval_ds = SemanticSegmentationDataset(
    image_processor=image_processor,
    txt_file=eval_txt,
    img_dir=img_dir,
    masks_dir=masks_dir,
    target_size=(100, 100)
)

In [27]:
train_dataloader = DataLoader(train_ds, batch_size=8, shuffle=True)
eval_dataloader = DataLoader(eval_ds, batch_size=8, shuffle=True)

In [28]:
from torch.optim import AdamW
from transformers import get_scheduler

# Set up optimizer and learning rate scheduler
optimizer = AdamW(
    model.parameters(),
    lr=2e-5,           # Learning rate for MaskFormer
    weight_decay=0.01  # Regularization
)

# Learning rate scheduler
num_training_steps = len(train_dataloader) * 200  # Assuming 200 epochs
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)


In [29]:
def custom_collate_fn(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])  # Batch pixel values
    labels = torch.stack([item["labels"] for item in batch])  # Batch labels (masks)
    return {"pixel_values": pixel_values, "labels": labels}

In [31]:
from torch.utils.data import DataLoader
from transformers import MaskFormerImageProcessor

# Initialize processor
image_processor = MaskFormerImageProcessor(ignore_index=255, reduce_labels=False)

# Initialize datasets (train.txt and test.txt)
train_dataset = SemanticSegmentationDataset(
    image_processor=image_processor,
    txt_file=train_txt,
    img_dir=img_dir,
    masks_dir=masks_dir,
    target_size=(100, 100)  # Resize to 100x100
)

eval_dataset = SemanticSegmentationDataset(
    image_processor=image_processor,
    txt_file=eval_txt,
    img_dir=img_dir,
    masks_dir=masks_dir,
    target_size=(100, 100)
)

# DataLoaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size=2,   # Set batch size
    shuffle=True,   # Shuffle for training
)

eval_dataloader = DataLoader(
    eval_dataset,
    batch_size=2,   # Set batch size
    shuffle=False   # No shuffle for evaluation
)

In [32]:
batch = next(iter(train_dataloader))
print("Keys in batch:", batch.keys())
print("Pixel Values Shape:", batch["pixel_values"].shape)  # (batch_size, 3, H, W)
print("Labels Shape:", batch["labels"].shape)              # (batch_size, H, W)

Keys in batch: dict_keys(['pixel_values', 'labels'])
Pixel Values Shape: torch.Size([2, 3, 100, 100])
Labels Shape: torch.Size([2, 100, 100])


In [37]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00


In [38]:
import evaluate

In [39]:
from torch.utils.tensorboard import SummaryWriter

# Define log directory for TensorBoard
log_path = './logs'
writer = SummaryWriter(log_path)

# Load the "mean_iou" metric using the evaluate library
metric = evaluate.load("mean_iou")


In [ ]:
import torch
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn.functional as F
from transformers import MaskFormerForInstanceSegmentation, MaskFormerImageProcessor

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load the MaskFormer model
model = MaskFormerForInstanceSegmentation.from_pretrained(
    "facebook/maskformer-swin-tiny-ade",
    num_labels=6,  # Adjust based on dataset
    ignore_mismatched_sizes=True
).to(device)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# Mixed Precision Training
scaler = GradScaler()

# Training Loop
model.train()
for epoch in range(200):
    print(f"Epoch {epoch}")
    curr_epoch_loss = []

    for idx, batch in enumerate(train_dataloader):
      pixel_values = batch["pixel_values"].to(device)
      mask_labels = batch["labels"].to(device)  # Shape: (batch_size, H, W)

      optimizer.zero_grad()

    # Mixed Precision Forward Pass
      with autocast(dtype=torch.float16):
        outputs = model(pixel_values=pixel_values)

        # Use masks_queries_logits for predicted masks
        mask_logits = outputs.masks_queries_logits  # Shape: (batch_size, num_queries, H, W)

        # Interpolate logits to match mask_labels size
        mask_logits = F.interpolate(
            mask_logits, size=mask_labels.shape[-2:], mode="bilinear", align_corners=False
        )

        # Ensure mask_logits has correct dimensions: (batch_size, num_classes, H, W)
        # This might require mapping the queries to classes (depends on your setup)
        mask_logits = mask_logits.permute(0, 2, 3, 1)  # Rearrange to (batch_size, H, W, num_queries)

        # Compute Cross-Entropy Loss (logits should not have an argmax applied)
        loss = F.cross_entropy(mask_logits, mask_labels)

        # Backward Pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()

        curr_epoch_loss.append(loss.item())
        if idx % 10 == 0:
            print(f"Step {idx}, Loss: {loss.item()}")

    # Print epoch loss
    print(f"Epoch {epoch}, Average Loss: {sum(curr_epoch_loss) / len(curr_epoch_loss)}")
    torch.cuda.empty_cache()

# Evaluation Loop
model.eval()
with torch.no_grad():
    eval_losses = []
    metric.reset()  # Reset the metric before evaluation (optional for some metrics)

    for batch in eval_dataloader:
        pixel_values = batch["pixel_values"].to(device)
        mask_labels = batch["labels"].to(device)

        with autocast(dtype=torch.float16):  # Enable mixed precision
            outputs = model(pixel_values=pixel_values)
            mask_logits = outputs.masks_queries_logits  # MaskFormer raw logits

            # Resize logits to match ground truth size
            mask_logits = F.interpolate(
                mask_logits, size=mask_labels.shape[-2:], mode="bilinear", align_corners=False
            )

            # Compute loss
            loss = F.cross_entropy(mask_logits, mask_labels)
            eval_losses.append(loss.item())

            # Convert logits to predicted class indices
            predictions = mask_logits.argmax(dim=1).cpu().numpy()  # (batch_size, H, W)
            references = mask_labels.cpu().numpy()  # Ground truth labels

            # Add batch predictions and references to the metric
            metric.add_batch(predictions=predictions, references=references)

    # Compute the final metrics
    results = metric.compute(num_labels=6, ignore_index=255)
    mean_iou = results["mean_iou"]
    mean_accuracy = results["mean_accuracy"]

    # Log results
    eval_loss_value = sum(eval_losses) / len(eval_losses)
    eval_loss.append(eval_loss_value)
    eval_iou.append(mean_iou)
    eval_acc.append(mean_accuracy)

    print(f"Average Evaluation Loss: {eval_loss_value}")
    print(f"Mean IoU: {mean_iou}")
    print(f"Mean Accuracy: {mean_accuracy}")

    writer.add_scalar("Eval/Loss", eval_loss_value, global_step=epoch)
    writer.add_scalar("Eval/Mean_IoU", mean_iou, global_step=epoch)
    writer.add_scalar("Eval/Mean_Accuracy", mean_accuracy, global_step=epoch)

model.train()  # Return to training mode

Using device: cuda


Some weights of MaskFormerForInstanceSegmentation were not initialized from the model checkpoint at facebook/maskformer-swin-tiny-ade and are newly initialized because the shapes did not match:
- class_predictor.bias: found shape torch.Size([151]) in the checkpoint and torch.Size([7]) in the model instantiated
- class_predictor.weight: found shape torch.Size([151, 256]) in the checkpoint and torch.Size([7, 256]) in the model instantiated
- criterion.empty_weight: found shape torch.Size([151]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 0


Step 0, Loss: 5.830498218536377
Step 10, Loss: 6.560971736907959
Step 20, Loss: 5.460800647735596
Step 30, Loss: 4.733295917510986
Epoch 0, Average Loss: 5.262798261642456
Epoch 1
Step 0, Loss: 4.204943656921387
Step 10, Loss: 4.216634750366211
Step 20, Loss: 4.302879810333252
Step 30, Loss: 4.832705020904541
Epoch 1, Average Loss: 4.356055777413505
Epoch 2
Step 0, Loss: 4.720031261444092
Step 10, Loss: 5.50396203994751
Step 20, Loss: 5.256143569946289
Step 30, Loss: 4.309160232543945
Epoch 2, Average Loss: 4.492826727458409
Epoch 3
Step 0, Loss: 4.471733093261719
Step 10, Loss: 3.9888269901275635
Step 20, Loss: 3.893688201904297
Step 30, Loss: 4.678355693817139
Epoch 3, Average Loss: 4.368869045802525
Epoch 4
Step 0, Loss: 4.423816204071045
Step 10, Loss: 5.889927387237549
Step 20, Loss: 4.688224792480469
Step 30, Loss: 3.8095500469207764
Epoch 4, Average Loss: 4.391020052773612
Epoch 5
Step 0, Loss: 4.164350986480713
Step 10, Loss: 4.455660343170166
Step 20, Loss: 5.045419692993164
S

In [ ]:
!kill -9 14330

In [174]:
!nvidia-smi

Tue Dec 17 00:05:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              33W /  70W |  15101MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--